## **Librerías a usar en la extracción de los pares del PDF**

In [1]:
import pdfplumber
import re
import os
import csv
import shutil

## Funciones para extraer conversaciones

In [2]:
def extraer_conversaciones_a_txt(ruta_pdf):
    # 1. Crear carpeta para guardar los archivos si no existe
    carpeta_salida = "data/raw/conversaciones_inicial"
    os.makedirs(carpeta_salida, exist_ok=True)

    # 2. Configuración
    # Páginas 13 a 146
    pagina_inicio = 12 
    pagina_fin = 146 
    
    # Patrón para detectar el INICIO de una nueva conversación
    # Busca números seguidos de ° o º (ej: "1° -", "2º -", "40° -")
    patron_inicio_capitulo = re.compile(r"^\d+\s*[°º]")
    
    # Buffer para ir guardando las líneas de la conversación actual
    conversacion_actual = []
    contador_conversaciones = 0

    print(f"Abriendo PDF: {ruta_pdf}...")

    with pdfplumber.open(ruta_pdf) as pdf:
        # Iteramos solo sobre las páginas indicadas
        # Usamos slice [pagina_inicio : pagina_fin] para ir directo al grano
        paginas_interes = pdf.pages[pagina_inicio:pagina_fin]
        
        print(f"Procesando {len(paginas_interes)} páginas...")

        for pagina in paginas_interes:
            texto = pagina.extract_text()
            if not texto:
                continue
            
            lineas = texto.split('\n')
            
            for linea in lineas:
                linea = linea.strip()
                
                # Detectamos si la línea es el inicio de una NUEVA conversación
                if patron_inicio_capitulo.match(linea):
                    
                    # Si ya teníamos una conversación acumulándose, la guardamos antes de empezar la nueva
                    if conversacion_actual:
                        guardar_archivo(carpeta_salida, contador_conversaciones, conversacion_actual)
                    
                    # Reiniciamos para la nueva conversación
                    conversacion_actual = []
                    contador_conversaciones += 1
                    conversacion_actual.append(linea) # Añadimos el título como primera línea
                
                else:
                    # Si no es título nuevo, es contenido de la conversación actual
                    # (Solo guardamos si ya hemos encontrado al menos el primer título)
                    if contador_conversaciones > 0:
                        conversacion_actual.append(linea)

        # 3. Guardar la última conversación que quedó en memoria al terminar el loop
        if conversacion_actual:
             guardar_archivo(carpeta_salida, contador_conversaciones, conversacion_actual)

    print("-" * 30)
    print(f"¡Proceso completado! Se crearon {contador_conversaciones} archivos en la carpeta '{carpeta_salida}'.")

def guardar_archivo(carpeta, numero, lineas):
    """Función auxiliar para escribir el archivo .txt"""
    nombre_archivo = f"conversacion_{numero}.txt"
    ruta_completa = os.path.join(carpeta, nombre_archivo)
    
    contenido = "\n".join(lineas)
    
    with open(ruta_completa, 'w', encoding='utf-8') as f:
        f.write(contenido)
    print(f"Guardado: {nombre_archivo}")

In [3]:
# Extraemos conversaciones
archivo_pdf = 'data/raw/book/book.pdf'
extraer_conversaciones_a_txt(archivo_pdf)

Abriendo PDF: data/raw/book/book.pdf...
Procesando 134 páginas...
Guardado: conversacion_1.txt
Guardado: conversacion_2.txt
Guardado: conversacion_3.txt
Guardado: conversacion_4.txt
Guardado: conversacion_5.txt
Guardado: conversacion_6.txt
Guardado: conversacion_7.txt
Guardado: conversacion_8.txt
Guardado: conversacion_9.txt
Guardado: conversacion_10.txt
Guardado: conversacion_11.txt
Guardado: conversacion_12.txt
Guardado: conversacion_13.txt
Guardado: conversacion_14.txt
Guardado: conversacion_15.txt
Guardado: conversacion_16.txt
Guardado: conversacion_17.txt
Guardado: conversacion_18.txt
Guardado: conversacion_19.txt
Guardado: conversacion_20.txt
Guardado: conversacion_21.txt
Guardado: conversacion_22.txt
Guardado: conversacion_23.txt
Guardado: conversacion_24.txt
Guardado: conversacion_25.txt
Guardado: conversacion_26.txt
Guardado: conversacion_27.txt
Guardado: conversacion_28.txt
Guardado: conversacion_29.txt
Guardado: conversacion_30.txt
Guardado: conversacion_31.txt
Guardado: con

### Función para extraer la parte hablada

In [4]:
def limpiar_conversaciones():
    carp_origen = "data/raw/conversaciones_inicial"
    carp_destino = "data/raw/conversaciones_final"
    
    # Textos a eliminar
    frases_prohibidas = [
        "conversaciones en aimara",
        "aymara aruskipawinaka"
    ]
    separador_clave = "-------------------&-------------------"

    # Crear carpeta de destino si no existe
    os.makedirs(carp_destino, exist_ok=True)
        
    print(f"Limpiando archivos de '{carp_origen}' a '{carp_destino}'...")

    # Iterar sobre cada archivo en la carpeta origen
    archivos = sorted([f for f in os.listdir(carp_origen) if f.endswith(".txt")])
    
    for nombre_archivo in archivos:
        ruta_origen = os.path.join(carp_origen, nombre_archivo)
        ruta_destino = os.path.join(carp_destino, nombre_archivo)
        
        with open(ruta_origen, 'r', encoding='utf-8') as f:
            lineas = f.readlines()
            
        lineas_limpias = []
        encontrado_separador = False
        
        for linea in lineas:
            linea_stripped = linea.strip()
            
            # 1. Lógica del Separador
            # Si aún no encontramos el separador, revisamos si esta línea lo es
            if not encontrado_separador:
                if separador_clave in linea:
                    encontrado_separador = True
                continue # Saltamos todo lo que esté antes (y la línea del separador también)
            
            # --- A partir de aquí estamos DESPUÉS del separador ---
            
            # 2. Eliminar líneas vacías
            if not linea_stripped:
                continue
                
            # 3. Eliminar líneas que solo tienen números (números de página)
            if linea_stripped.isdigit():
                continue
                
            # 4. Eliminar líneas con frases prohibidas (Encabezados)
            if any(frase in linea.lower() for frase in frases_prohibidas):
                continue
            
            # Si pasa todos los filtros, la guardamos
            lineas_limpias.append(linea)
            
        # Guardar el archivo limpio
        if lineas_limpias:
            with open(ruta_destino, 'w', encoding='utf-8') as f_out:
                f_out.writelines(lineas_limpias)
                
    print(f"¡Listo! Se han procesado {len(archivos)} conversaciones.")
    print(f"Revisa la carpeta: {carp_destino}")

In [5]:
# Extraemos la parte hablada
limpiar_conversaciones()

Limpiando archivos de 'data/raw/conversaciones_inicial' a 'data/raw/conversaciones_final'...
¡Listo! Se han procesado 40 conversaciones.
Revisa la carpeta: data/raw/conversaciones_final


In [6]:
# Eliminanos carpeta
shutil.rmtree("data/raw/conversaciones_inicial")

### Función para reconstruir lo hablado por un hablante

In [7]:
def reconstruir_lineas_conversacion():
    # Carpetas de entrada y salida
    carpeta_origen = "data/raw/conversaciones_final"
    carpeta_destino = "data/raw/conversaciones_finales"
    
    if not os.path.exists(carpeta_destino):
        os.makedirs(carpeta_destino)
    
    # Patrón para identificar el inicio de un turno:
    # ^       -> Inicio de línea
    # [A-Z]{1,2} -> Una o dos letras mayúsculas (ej: M, A, WC)
    # \)      -> Un paréntesis de cierre literal
    patron_hablante = re.compile(r"^[A-Z]{1,2}\)")

    print(f"Procesando archivos de '{carpeta_origen}'...")
    
    archivos = sorted([f for f in os.listdir(carpeta_origen) if f.endswith(".txt")])

    for nombre_archivo in archivos:
        ruta_origen = os.path.join(carpeta_origen, nombre_archivo)
        ruta_destino = os.path.join(carpeta_destino, nombre_archivo)
        
        with open(ruta_origen, 'r', encoding='utf-8') as f:
            lineas_crudas = f.readlines()
        
        lineas_reconstruidas = []
        buffer_turno = "" # Aquí acumularemos el texto de un turno completo
        
        for linea in lineas_crudas:
            linea = linea.strip()
            if not linea: continue # Saltar líneas vacías
            
            # Verificamos si la línea actual es el INICIO de un hablante
            es_nuevo_turno = patron_hablante.match(linea)
            
            if es_nuevo_turno:
                # Si ya teníamos un turno acumulado en el buffer, lo guardamos en la lista final
                if buffer_turno:
                    lineas_reconstruidas.append(buffer_turno)
                
                # Empezamos el nuevo turno
                buffer_turno = linea
            else:
                # Si NO es un nuevo turno, es la continuación del anterior (la frase cortada)
                # Lo unimos al buffer con un espacio
                if buffer_turno:
                    buffer_turno += " " + linea
                else:
                    # Caso borde: si el archivo empieza con una línea sin hablante (raro tras la limpieza)
                    buffer_turno = linea
        
        # IMPORTANTE: Guardar el último turno que quedó en el buffer al terminar el bucle
        if buffer_turno:
            lineas_reconstruidas.append(buffer_turno)
            
        # Escribir el archivo reconstruido
        with open(ruta_destino, 'w', encoding='utf-8') as f_out:
            f_out.write('\n'.join(lineas_reconstruidas))

    print(f"¡Proceso terminado! Se han reconstruido {len(archivos)} conversaciones.")
    print(f"Los archivos listos están en: {carpeta_destino}")



In [8]:
# reconstruimos conversaciones
reconstruir_lineas_conversacion()

Procesando archivos de 'data/raw/conversaciones_final'...
¡Proceso terminado! Se han reconstruido 40 conversaciones.
Los archivos listos están en: data/raw/conversaciones_finales


In [9]:
# Eliminamos carpeta
shutil.rmtree("data/raw/conversaciones_final")

### Función para crear corpus paralelo

In [11]:
def generar_corpus_final():
    # Configuración de rutas
    carpeta_entrada = "data/raw/conversaciones_finales"
    carpeta_salida = "data/raw/book"
    os.makedirs(carpeta_salida, exist_ok=True)
    archivo_salida = os.path.join(carpeta_salida, "book.tsv")
    
    # Conjunto para guardar pares únicos (ayuda a eliminar duplicados automáticamente)
    corpus_unico = set()
    
    # Compilamos el regex para quitar el hablante (Ej: "M) ", "R) ", "AB) ")
    # Busca al inicio de línea (^), 1 o 2 letras mayúsculas, un paréntesis y espacios opcionales
    patron_hablante = re.compile(r"^[A-Z]{1,2}\)\s*")
    
    print(f"Leyendo archivos de '{carpeta_entrada}'...")
    
    # Listar archivos
    if not os.path.exists(carpeta_entrada):
        print(f"Error: No existe la carpeta {carpeta_entrada}")
        return

    archivos = [f for f in os.listdir(carpeta_entrada) if f.endswith(".txt")]
    lineas_ignoradas_por_slash = 0
    
    for nombre_archivo in archivos:
        ruta = os.path.join(carpeta_entrada, nombre_archivo)
        
        with open(ruta, 'r', encoding='utf-8') as f:
            for linea in f:
                linea = linea.strip()
                if not linea: continue
                
                # REGLA 1: Verificar que tenga EXACTAMENTE un '/'
                if linea.count('/') != 1:
                    print("Linea rara --> ",linea)
                    lineas_ignoradas_por_slash += 1
                    continue
                
                # Dividir en Aimara (izquierda) y Español (derecha)
                partes = linea.split('/')
                parte_aimara_bruta = partes[0].strip()
                parte_espanol_bruta = partes[1].strip()
                
                # REGLA 2: Quitar el identificador del hablante del lado Aimara
                # Reemplazamos "M) Hola" por "Hola"
                aimara_limpio = patron_hablante.sub('', parte_aimara_bruta)
                
                # REGLA 3: Convertir a minúsculas
                aimara_final = aimara_limpio.lower().strip()
                espanol_final = parte_espanol_bruta.lower().strip()
                
                # Validación extra: que no hayan quedado vacíos
                if aimara_final and espanol_final:
                    # REGLA 4: Al añadir a un set, se eliminan duplicados automáticamente
                    corpus_unico.add((aimara_final, espanol_final))

    # Ordenamos alfabéticamente para que el archivo quede ordenado
    lista_corpus = sorted(list(corpus_unico))
    
    # Escribir el archivo TSV
    print(f"Escribiendo {len(lista_corpus)} pares únicos en '{archivo_salida}'...")
    
    with open(archivo_salida, 'w', encoding='utf-8', newline='') as f_out:
        # Usamos tabulador como separador
        writer = csv.writer(f_out, delimiter='\t')
        
        # Encabezado (opcional, útil para Pandas)
        writer.writerow(['aymara', 'spanish'])
        
        # Escribir datos
        writer.writerows(lista_corpus)
        
    print("-" * 30)
    print("¡Proceso Finalizado!")
    print(f"Total de pares únicos generados: {len(lista_corpus)}")
    print(f"Líneas descartadas por tener múltiples '/' o ninguno: {lineas_ignoradas_por_slash}")


In [12]:
# Creamos corpus paralelo
generar_corpus_final()

Leyendo archivos de 'data/raw/conversaciones_finales'...
Linea rara -->  Una mañana en la Terminal de Buses
Linea rara -->  M) Ch’uqimpi ullukumpi, ch’arkhimpi, wali suma ulluk kaltu phayiristha / Con la papa, la papalisa y el charque, puedo cocinar un rico caldo de papalisa B: Kuna manq’as markaman wali uñyt’ata? / ¿Qué comida es la más conocida en tu pueblo?
Linea rara -->  M) Kuna pachaxis jilata? / ¿Qué hora ya es hermano? B: Kimsaqalqu alwa chikatanixiwa / Son las 08:30 de la mañana A: Pachaxiw jilata, jutir phaxsin aruskiparakiñani, jichhax qulliriwkaru sarä, p’iqimpi, laka ch’akhampiw usutu / Ya es hora hermano, conversaremos el próximo mes, ahora tengo que ir donde el médico, me duele la cabeza y mi diente
Linea rara -->  J) Jupax kuna luririsa? / ¿Cuál es la ocupación de él? A: Jupax jakhuriwa / Él es contador
Linea rara -->  R) Nayax yatiqiritwa. Qhawqha jilatanitasa? Yo soy estudiante. ¿Cuántos hermanos tienes?
Linea rara -->  R) Jumax kuna manq's munta? / ¿Qué comida quiere

In [13]:
shutil.rmtree("data/raw/conversaciones_finales") # eliminamos carpeta

## **Preprocesaiento del texto**

In [14]:
import pandas as pd
import unicodedata
import re
import csv
import os

In [15]:
df = pd.read_csv("data/raw/book/book.tsv", sep="\t", encoding="utf-8")
df.tail()

,aymara,spanish
1620,"yuspajara, sarxa","gracias, me voy"
1621,¡suma muxsarakisä uka manq’axa!,que rico es esa comida!
1622,¿juman sutimax kunasa?,¿cuál es tu nombre?
1623,¿kullaka kunas puqhat sutimaxa?,¿cuál es tu nombre completo hermana?
1624,ñanqha uruwa,es viernes


In [16]:
# Caracteres únicos del texto en Español
spanish_chars = sorted(set("".join(df["spanish"].str.lower())))
print(spanish_chars)
print("Cantidad de caracteres en español:", len(spanish_chars))

[' ', '!', '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '¡', '´', '¿', 'á', 'é', 'í', 'ñ', 'ó', 'ú', '’', '…']
Cantidad de caracteres en español: 56


In [17]:
# Caracteres únicos del texto en Aymara
aymara_chars = sorted(set("".join(df["aymara"].str.lower())))
print(aymara_chars)
print("Cantidad de caracteres en aymara:", len(aymara_chars))

[' ', '!', "'", ',', '.', '3', '5', ':', '?', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '¡', '´', '¿', 'á', 'ä', 'é', 'í', 'ï', 'ñ', 'ó', 'ú', 'ü', '’', '…']
Cantidad de caracteres en aymara: 50


In [18]:
def normalize_unicode(text: str) -> str:
    '''Para asegurarnos de que los caracteres se representen de una forma estándar interna'''
    return unicodedata.normalize("NFC", text)

def normalize_quotes(text: str) -> str:
    '''Para normalizar las comillas a comillas simples estándar ASCII'''
    text = text.replace("«", "\"").replace("»", "\"")
    text = text.replace("“", "\"").replace("”", "\"")
    text = text.replace("‘", "'").replace("’", "'")
    return text

def normalize_punctuation(text: str) -> str:
    '''Para normalizar signos de puntuación especiales'''
    text = text.replace("—", "-")
    text = text.replace("…", "...")
    return text

def clean_brackets(text: str) -> str:
    '''Para eliminar los corchetes [ y ] del texto'''
    return text.replace("[", "").replace("]", "")

def remove_verse_numbers(text: str) -> str:
    '''Cualquier número en el texto hace referencia a un código de versículo'''
    words = text.split()
    cleaned_words = [w for w in words if not any(ch.isdigit() for ch in w)] # Mantenemos solo palabras que no contienen dígitos
    return " ".join(cleaned_words)

def clean_spaces(text: str) -> str:
    '''Para normalizar los espacios en el texto'''
    text = re.sub(r"\s+", " ", text)
    return text.strip()

def lowercase(text: str) -> str:
    '''Para convertir todo el texto a minúsculas'''
    return text.lower()

def preprocess(text: str) -> str:
    text = normalize_unicode(text)
    text = normalize_quotes(text)
    text = normalize_punctuation(text)
    text = clean_brackets(text)
    # text = remove_verse_numbers(text) # Mantenemos los números
    text = clean_spaces(text)
    text = lowercase(text)
    return text

### Aplicamos preprocesamiento a book.tsv para obtener book/spanish.txt y book/aymara.txt

In [19]:
raw_corpus = "data/raw/book/book.tsv"
folder_clean_path = "data/clean/book"
os.makedirs(folder_clean_path, exist_ok=True)
output_spanish_clean = f"{folder_clean_path}/spanish.txt"
output_aymara_clean = f"{folder_clean_path}/aymara.txt"

spanish_lines = []
aymara_lines = []

with open(raw_corpus, "r", encoding="utf-8") as f:
    reader = csv.DictReader(f, delimiter="\t")
    
    for row in reader:
        spanish_raw = row["spanish"]
        aymara_raw = row["aymara"]

        spanish_clean = preprocess(spanish_raw)
        aymara_clean = preprocess(aymara_raw)

        if not spanish_clean or not aymara_clean:
            continue
        
        spanish_lines.append(spanish_clean)
        aymara_lines.append(aymara_clean)

# Guardamos los resultados
with open(output_spanish_clean, "w", encoding="utf-8") as f:
    for line in spanish_lines:
        f.write(line + "\n")

with open(output_aymara_clean, "w", encoding="utf-8") as f:
    for line in aymara_lines:
        f.write(line + "\n")

print(f"Archivos {output_spanish_clean} y {output_aymara_clean} generados exitosamente...")

Archivos data/clean/book/spanish.txt y data/clean/book/aymara.txt generados exitosamente...
